<a href="https://colab.research.google.com/github/aysegulguzel/Generative-AI-Engineering/blob/main/GenAI_RAG_ChromaDB_HuggingFace_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install chromadb==0.3.21 tiktoken==0.3.3 pydantic==1.9.0 beautifulsoup4 requests openai==0.28

In [6]:
from bs4 import BeautifulSoup

# Example HTML snippet
html_snippet = '<a class="hover:underline" target="_self" href="/dataaisummit/session/bayers-alyce-advancing-drug-development-data-intelligence">Bayer\'s ALYCE: Advancing Drug Development with Data Intelligence</a>'

# Parse HTML using BeautifulSoup
soup = BeautifulSoup(html_snippet, 'html.parser')

# Extract title and URL
title = soup.get_text().strip()


print("Title:", title)



Title: Bayer's ALYCE: Advancing Drug Development with Data Intelligence


In [7]:
import requests
from bs4 import BeautifulSoup
import csv

# Function to extract session data from a page
def extract_session_data(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, "html.parser")
    anchor_tags = soup.find_all("a", class_="hover:underline")
    session_data = []
    for anchor in anchor_tags:
        title = anchor.get_text().strip()
        url = "https://www.databricks.com" + anchor["href"]
        session_data.append([title, url])
    return session_data

# Base URL of the webpage containing the sessions
base_url = "https://www.databricks.com/dataaisummit/agenda?page="

# List to store all session data
all_session_data = []

# Iterate through pages
for page_num in range(1, 9):  # Assuming there are 8 pages
    page_url = base_url + str(page_num)
    session_data = extract_session_data(page_url)
    all_session_data.extend(session_data)

# Write the extracted data to a CSV file
with open("dataaisummit_agenda.csv", "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Title", "URL"])
    writer.writerows(all_session_data)

print("Data extraction completed.")


Data extraction completed.


In [8]:
#prepare data
# Read the CSV file into a DataFrame
import pandas as pd
dais_pdf = pd.read_csv("dataaisummit_agenda.csv")

# Display the DataFrame
display(dais_pdf)


,Title,URL
0,84.51’s Journey to Unlock Data Value with Data...,https://www.databricks.com/dataaisummit/sessio...
1,A Lean Data Factory: The Power of a Data Intel...,https://www.databricks.com/dataaisummit/sessio...
2,A Modern Approach to Dimensional Modeling - In...,https://www.databricks.com/dataaisummit/sessio...
3,Accelerating JP Morgan Data Product Developmen...,https://www.databricks.com/dataaisummit/sessio...
4,Accelerating Operational Excellence with Gener...,https://www.databricks.com/dataaisummit/sessio...
...,...,...
139,Automating Government Transparency in Declassi...,https://www.databricks.com/dataaisummit/sessio...
140,Bayer's ALYCE: Advancing Drug Development with...,https://www.databricks.com/dataaisummit/sessio...
141,Beyond Text-Based Transformers,https://www.databricks.com/dataaisummit/sessio...
142,Blueprint: How Unilever Utilizes Metadata to P...,https://www.databricks.com/dataaisummit/sessio...


In [9]:
dais_pdf["full_text"] = dais_pdf.apply(
    lambda row: f"""Title: {row["Title"]}""".strip(),
    axis=1,
)
print(dais_pdf.iloc[0]["full_text"])

Title: 84.51’s Journey to Unlock Data Value with Databricks Unity Catalog


In [10]:
texts = dais_pdf["full_text"].to_list()

In [ ]:
#set up chroma and create collection

In [11]:
import chromadb
from chromadb.config import Settings

# Specify the directory to store the database
persist_directory = "/content/user_db"

# Initialize chromadb client
chroma_client = chromadb.Client(
    Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=persist_directory
    )
)


In [12]:
#Assign the value of my_talks to the collection_name variable
collection_name = "my_talks"

# If you have created the collection before, you need delete the collection first
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)

print(f"Creating collection: '{collection_name}'")
talks_collection = chroma_client.create_collection(name=collection_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Creating collection: 'my_talks'


In [13]:
#add data
talks_collection.add(
    documents=texts,
    ids=[f"id{x}" for x in range(len(texts))]
    )

In [14]:
#query data
import json

results = talks_collection.query(
    query_texts=["language models"],
    n_results=10)

print(json.dumps(results, indent=4))

{
    "ids": [
        [
            "id2",
            "id20",
            "id38",
            "id56",
            "id74",
            "id92",
            "id110",
            "id128",
            "id15",
            "id33"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Database",
            "Title: A Modern Approach to Dimensional Modeling - In a Columnar Data

In [15]:
#create a model pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import os

# Specify the desired cache directory
cache_dir = "/path/to/your/cache/directory"

# Pick a model from HuggingFace that can generate text
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(
    model_id, cache_dir=cache_dir
)  # Use a pre-cached model
lm_model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=cache_dir)

pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device_map="auto",
    handle_long_generation="hole",
)


In [16]:
#primpt engineering for question answering
# Come up with a question that you need the LLM assistant to help you with
# A sample question is "Help me find sessions related to XYZ"
# Note: Your "XYZ" should be related to the query you passed in Question 3.
question = "Help me find sessions related to language models"

# Provide all returned similar documents from the cell above below
context = " ".join([f"#{i}" for i in results["documents"][0]])

# Feel free to be creative how you construct the prompt. You can use the demo notebook as a jumpstart reference.
# You can also provide more requirements in the text how you want the answers to look like.
# Example requirement: "Recommend top-5 relevant sessions for me to attend."
prompt_template = f"""You will help users plan their conference titled "Data + AI Summit". Here are your task requirements:
- Convert the list into a conference plan organized by session title.
- Recommend top 5 relevant sessions for the user to attend.
- Always include a brief 2-sentence summary for each session.
- Format the planning using markdown.

Relevant talks:
{context}

What the user is looking for: {question}

DATA + AI Summit plan:"""

In [17]:
#generate response
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You will help users plan their conference titled "Data + AI Summit". Here are your task requirements:
- Convert the list into a conference plan organized by session title.
- Recommend top 5 relevant sessions for the user to attend. 
- Always include a brief 2-sentence summary for each session. 
- Format the planning using markdown.

Relevant talks:
#Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: A Modern Approach to Dimensional Modeling - In a Columnar Database #Title: Beyond Text-Based Transformers #Title: Be

Use OpenAI models for RAG

In [20]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR-API-HERE"

In [21]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [22]:
 import tiktoken

price_token = 0.002
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
cost_to_run = len(encoder.encode(prompt_template)) / 1000 * price_token
print(f"It would take roughly ${round(cost_to_run, 5)} to run this prompt")

It would take roughly $0.00048 to run this prompt


In [23]:
gpt35_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt_template},
    ],
    temperature=0,  # 0 makes outputs deterministic; The closer the value is to 1, the more random the outputs are for each time you re-run.
)

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
print(gpt35_response.choices[0]["message"]["content"])

In [ ]:
gpt35_response["usage"]["total_tokens"]